# Genome assembly and quality control of assembly

## Introduction 

After the previous steps of quality control (QC), we have reads still in `.fastq` format, but now we have a summary of their sequencing quality. Furthermore, we have removed regions with poor confidence in base calling (where we cannot be sure if the assigned nucleotides are right) and we removed the adaptor sequences used by the sequencing machine to amplify the DNA. 

In this series of steps, we will assemble the reads using a tool called `shovill`. Once again, we will mimic how to run the commands in the **Compute Canada (CC)** cluster of analysis. 

### Required tools and environment set up

For these tutorials, tools will be made available using singularity containers, which can be run using the command `singularity run tool_image`. These tools have been made available in the environment already, so there is no need to download them.

Tools used in this tutorial:
- `Shovill         v1.1.0 (with SPAdes v3.14.1)`
- `Quast           v5.0.2`
- `Singularity     v3.10.2`
- `CheckM          v1.2.2`

Tools downloaded for the tutorial are in the a shared folder and in the `tutorials` directory are the results of our commands. 

**Notes:** 
- After opening this notebook, make sure that you have the **bash** kernel selected. 
- Try to run all the commands from your home or base directory, to get there just use `cd`

Also, run the source command to make all pre-packed software available in your environment.

In [ ]:
# source PATH to use module function
source /cvmfs/soft.computecanada.ca/config/profile/bash.sh

Explore the results produced so far using the `tree` command, and get a sense of the structure for the software folder.

In [ ]:
# get to your HOME directory
cd

echo -e "Structure of tutorials folder:"
tree -dL 2 $HOME/tutorials

echo -e "Structure of software directory:"
tree -dL 2 /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto

For downstream assembly, we will use the curated reads contained in the `trimmed_reads` subdirectory (20 files containing paired reads for 10 isolates of _P. aeruginosa_). 

In [ ]:
cd
ls -1 $HOME/tutorials/trimmed_reads

## De-novo assembly with Shovill

Shovill is a tool that optimizes the assembler `Spades` to minimize the run time, while maintaining the quality of results. It generates a draft genome using heuristic algorithms and does not require a reference genome to guide the process. See the GitHub repositories of [shovill](https://github.com/tseemann/shovill) and [SPAdes](https://github.com/ablab/spades) for more details. 

Shovill is not available as a module pre-installed in **CC**, so we must use another strategy. The easiest one is to use a container, we can install a **Docker** container, but Docker containers are not suitable for high performance clusters like **Compute Canada** because they are used mostly when you have administrator privileges. Thus, many HPC allow use of **Singularity** images as an alternative (For more info about what is containerization you can read https://www.melbournebioinformatics.org.au/tutorials/tutorials/docker/docker/).

A useful repository of **Singularity** images is located at https://depot.galaxyproject.org/singularity/

<font color='darkred'>_**Notes for compute canada:**_ </font>  
- Singularity needs to be loaded into the system. On the CIDGOH servers, it is loaded by default. Run the following code to have singularity available in your compute canada session. 

>    module load singularity

- We have already downloaded the **Shovill** singularity container. In CC, you may need to do it, so run the following command pull it from the repository into your local directory. The command tells the system to pull a container from a repository into your local system.

>    singularity pull shovill_1.1.sif https://depot.galaxyproject.org/singularity/shovill%3A1.1.0--hdfd78af_1


In [ ]:
# executing shovill

singularity exec /mnt/cidgoh-object-storage/images/shovill_1.1.sif shovill --help

### What does Shovill do?

1. Unifies coverage depth (how many times is a region covered by reads on average) for all genomes
2. Trims adapters and poor quality reads if necessary
3. Assembles using SPAdes
4. Polishes genomes (improves quality) and filters low quality contigs


In [ ]:
# create output directory
CONTIGS_DIR="$HOME/tutorials/results/contigs"

# define PATH to trimmed_reads
TRIMMED_READS="$HOME/tutorials/trimmed_reads"

To execute shovill, we run the command from the singularity container we just downloaded. Genome assembly is the most resource intensive process in the pipeline, so it will probably take a while to run (20 - 30 min). As input, we will use the `trimmed_reads` files and only two isolates. The remaining ones are already assembled and can be copied into our environment later.

<font color='darkred'>_**Notes for compute canada:**_ </font>  
- Allocate sufficiente memory as the size of every genome must be kept in storage while it is assembled
- Bioinformatic procedures usually use multiple threads to optimize performance, so their efficiency increases with the number of available cores (including **SPAdes**). 
- In shovill, the `--ram` option specifies the available ram per thread (core)
    - Spades will take input of RAM from shovill as total available mem, better to input limit manually with `--opts "-m XX"`

In [ ]:
# for loop to run a command for each sample

for READ1 in $(ls $TRIMMED_READS/*R1.fastq.gz | head -n 2)
do

    
    READ2=${READ1/_R1/_R2}                                                              # substitute R1 for R2 in variable
    PREFIX_ISOLATE=$(basename $READ1 _R1.fastq.gz)                                      # create variable with isolate name only
    echo "Started processing $PREFIX_ISOLATE"
    
    singularity exec /mnt/cidgoh-object-storage/images/shovill_1.1.sif shovill  \
        --R1 $READ1                                                                     `# specify paired read 1` \
        --R2 $READ2                                                                     `# specify paired read 2` \
        --outdir $CONTIGS_DIR                                                           `# define output directory` \
        --force                                                                         `# overwrite results if already available` \
        --ram 140                                                                       `# how much ram memory to use`
    
    mv "$CONTIGS_DIR/contigs.fa" $CONTIGS_DIR/${PREFIX_ISOLATE}_contigs.fa
    
    echo "Finished assembly of sample $PREFIX_ISOLATE"
    
done

In this tutorial, we optimized the runtime by processing only two samples. The remaining assemblies can be imported in the same directory for future steps using the command below.

In [ ]:
cp /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/precomputed_results/contigs/* $HOME/tutorials/results/contigs


The main output of the **Shovill** pipeline are the files ending in `contigs.fa` which contain assembled reads in fasta format. We can verify that this format contains a header for every contig and then the reads.  

More information about the `.fasta` format can be found in this [wikipedia page.](https://en.wikipedia.org/wiki/FASTA_format)

In [ ]:
head "$CONTIGS_DIR/ERR10479510_contigs.fa"

As a refresher, this is the current structure of directories for our `tutorials` project folder.

In [ ]:
tree -dL 2 $HOME/tutorials

## Quality control of draft genomes

**Shovill** produces contigs (overlapping consensus regions of DNA) for every isolate. However, we may have contaminated cultures growing other bacteria besides our organism of interest. Also, given the non-targeted approach of sequencing, the reads from an isolate may have poor quality (low reliability in base calling or poor coverage of certain regions) resulting in inadequate assemblies.

Thus, after producing draft genomes, we typically conduct additional checks to verify the characteristics of the resulting files and make sure that we do not have contamination in our samples. 

### General expectation

- The average size of contigs is over 5000 base pairs 
- No more than 1,000 should be produced in total 
- The resulting assembly should have a coverage of at least 90% of the reference genome

### Quast

Quast [(github:quast)](https://github.com/ablab/quast) produces quantitative summaries of the contigs in every assembly. If we specify a reference genome for the organism, it can evaluate misassemblies, unaligned contigs, and overall coverage against the reference genome. 

**_Some metrics include:_** 
- Number of contigs and number of contigs > 500bp
- **N50** or the length at which the collection of all contigs of at least that length covers half of the assembly 
- **NG50** is similar to **N50** but measures the coverage of the reference genome
- Number of misassemblies including inversions, relocations, and translocations
- Number and total length of unaligned contigs (against the reference genome)

As the data we are analyzing in the tutorial comes from _P. aeruginosa_ isolates ([PMID:34412676](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8376114/) - [BioProject:PRJEB56397](https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB56397)), we will use the [PAO1 reference strain](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCF_000006765.1/) for quality control. 

Reference genomes contain two files, a fasta sequence file [`.fna, .fa`] and an annotation file [`.gff`]. These reference files are stored in the software directory.

In [ ]:
# load quast into our environment
module load StdEnv/2020 gcc/9.3.0 quast/5.0.2

echo -e "Remember the structure of our software directory:

$(tree -dL 2 /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto)

And the contents of the reference_data subfolder are:

$(cd /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/ && ls)"

In [ ]:
# create ENV variable to input and output directory
CONTIGS_DIR="$HOME/tutorials/results/contigs"
RESULTS_QUAST="/$HOME/tutorials/results/assembly_quast"

The main command `quast.py` produces several reports, with formats such as `.pdf, .html, and .csv`, containing the previously mentioned metrics. 

They can be opened directly in Jupyter by clicking the file on the explorer or exported to your local computer for further visualization. 

In [ ]:
quast.py $CONTIGS_DIR/*contigs.fa                                                                                       `# pattern for contig files produced by shovill` \
    -r /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/GCF_000006765.1_ASM676v1_genomic.fna.gz       `# reference genome` \
    -g /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/GCF_000006765.1_ASM676v1_genomic.gff.gz                                     `# reference genomic features positions` \
    -o $RESULTS_QUAST                                                                                                  `# output directory` \
    --threads 12

We can preview metrics such as the N50 of the assemblies and the coverage of the reference genome using the commands below. 

In [ ]:
echo -e "These files are the output of QUAST:"
ls -1 $RESULTS_QUAST

# overview of N50 and coverage
echo
cat $RESULTS_QUAST/report.tsv                      `# print the file` | \
cut -f 1-5                                         `# cut the columns 1 to 5 (separated by tab)` | \
grep -E 'Assembly|N50|fraction'                    `# select lines matching the pattern Assembly OR N50 OR fraction` | \
column -ts $'\t'                                   `# print table separating at tabs ($'\t')`

### CheckM


**CheckM** infers the quality of the genome assembly based on the presence and uniqueness of sets of gene markers that are specific to species/taxa, and determines the completeness (coverage of reference genome) and the contamination of the input draft genomes.

**CheckM** is not available in the CC cluster. So, we use a singularity container with the latest version. For more info, visit https://github.com/Ecogenomics/CheckM. 


In [ ]:
# create output directory and set environment variable 
RESULTS_CHECKM="$HOME/tutorials/results/assembly_checkm"

1. The following command creates a dataset with specific genomic markers for a species, taxon or genus 

`checkm taxon_set <species/genus/taxon> <taxon_name> <marker_file>`

**Note:** some of the files used in the tutorials are in the `/mnt` shared drive/folder. Singularity may not be able to recognize that drive by default, so we run `export SINGULARITY_BIND="/mnt,/etc"` to make sure its readable.

In [ ]:
# add drives to singularity
export SINGULARITY_BIND="/mnt,/etc"

# command
singularity exec /mnt/cidgoh-object-storage/images/checkm_1.2.2.sif checkm \
    taxon_set species 'Pseudomonas aeruginosa' /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/pseudomonas.ms


2. Using `checkm analysis` we identify what taxon-specific marker sets are included in every assembly. The process for the samples used in the tutorial (n = 10) should take around 5 min. 

In [ ]:
singularity exec /mnt/cidgoh-object-storage/images/checkm_1.2.2.sif checkm analyze \
    /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/pseudomonas.ms           `#file with checkm marker set for assemblies` \
    $CONTIGS_DIR                                                                                `#dir with assemblies in fasta format` \
    $RESULTS_CHECKM                                                                             `#output directory` \
    -x fa                                                                                       `#extension of assemblies` \
    -t 10 

2. Then, with `checkm qa` we produce a summary of contamination results. This step should not take long (less than 1 min).

In [ ]:
singularity exec /mnt/cidgoh-object-storage/images/checkm_1.2.2.sif checkm qa                                                                 \
        /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/pseudomonas.ms       `#file with checkm marker set for assemblies` \
        $RESULTS_CHECKM                                                                        `#output directory` \
        --file $RESULTS_CHECKM/checkm_output.tsv                                                                    \
        --tab_table                                                                             `# print tabular output` \
        --threads 10                                                                            `# number of simultaneous threads for process` \
        --out_format 1                                                                          `# format of output 1 = summary, 2 = extended`

Now, we can review the output of results for contamination using **CheckM**. 

- *Completeness* is a measure of the coverage of gene marker sets spected for a species in a given contig. 
- *Contamination* shows the presence of multi-copy marker genes in the genome assembly. 
- *Strain heterogeneity* is determined by the number of multy-copy gene marker sets that have an amino acid identity >=  90%.

A high heterogeneity suggests that a majority of the contamination comes from closely related organisms. A smaller value may come from phylogenetically distinct sources

In [ ]:
column -ts $'\t' $RESULTS_CHECKM/checkm_output.tsv  